In [ ]:
"""Program to calculate Bray-Curtis similarity values between two datasets.  
"""
numprocess = 410
import math
from google.colab import drive
drive.mount('/content/gdrive')

#1. THE OUTFILE NAME.  
outname = '...path.../VC.Plus.3.Joined.BC.Matrix.csv'

#2. SPECIFY ALL THE INPUT METAGENOME FILES: 
VCfiles = ['...path.../Genus.Site1.Loop1.TRANSPOSED.csv',
         '...path.../Genus.Site2.Loop3.TRANSPOSED.csv',
         '...path.../Genus.Site3.Loop2.TRANSPOSED.csv',
         '...path.../Genus.Site4.Loop1.TRANSPOSED.csv',
         '...path.../Genus.Site5.Loop3.TRANSPOSED.csv',
         '...path.../Genus.Site6.Loop2.TRANSPOSED.csv']

altri =  ['...path.../A.K.Pri.4663797.3_organism_genus_hits.csv', #PRI
          '...path.../A.Z.USA.4.14_organism_genus_hits.csv.xls',  #ASB
          '...path.../A.Z.USA.1.23_organism_genus_hits.csv.xls']  #ANR

files = VCfiles  + altri 
nummetas =  len(files)
dataset = str(nummetas)

mglabel =['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'PRI', 'ASB', 'ANR']
print(mglabel)

stringmetas =[]
for x in range(nummetas):   # for reading files and converting each taxon to a list of [taxon, number]
  with open(files[x], 'r') as f:
    data = f.read()
    #print(repr(data)[:100])
    stringmetas.append(str(data))
  f.close()
print('Number of metagenomes =', len(stringmetas))
print( 'column labels =', mglabel)

cleanmetas =[]            # gets rid of unwanted text, converts number from string to integer
for x in range(nummetas):
  datastring = stringmetas[x].replace('unclassified (derived from ', '').replace(')', '').replace('Candidatus ', '')
  metalines = datastring.split('\n')
  metalines.pop()
  integermeta =[]
  for y in range(numprocess):  #len(metalines)):
    row = metalines[y].split(',')
    integermeta.append([row[0], int(row[1]), 0.0])
  cleanmetas.append(integermeta)

totalslist =[]             # counts the total number of taxa in each metagenome
for x in range(nummetas):
  count =0
  for y in range(numprocess): #len(cleanmetas[x])):
    count += cleanmetas[x][y][1]
  totalslist.append(count)
print('Total counts for each metagenone: ', totalslist)

for x in range(nummetas):
  for y in range(len(cleanmetas[x])):
     cleanmetas[x][y][2] = cleanmetas[x][y][1] / totalslist[x]

### need to make union of all genera
taxonset = set()
for x in range(nummetas):
  for y in range(numprocess):
    if cleanmetas[x][y][0] not in taxonset:
      taxonset.add(cleanmetas[x][y][0])

taxonlist = list(taxonset)
print('Total taxa in the union =', len(taxonlist))

### need to assign '0' values to missing taxa in each metagenome
for x in range(len(cleanmetas)):
  genera = set()
  for y in range(len(cleanmetas[x])):
    genera.add(cleanmetas[x][y][0])
  for z in range(len(taxonlist)):
    if taxonlist[z] not in genera:
      cleanmetas[x].append([taxonlist[z], 0, 0.0])

for x in range(len(cleanmetas)):
  cleanmetas[x].sort(key = lambda x: x[0])
  cleanmetas[x].reverse()

# make the BC matrix as a table
table = []
for x in range(len(cleanmetas)):                   # go through each meta (len = 7)
  row =[]                                         # list of BC values 
  for y in range(len(cleanmetas)):                  
    bcsum =0
    for z in range(len(taxonlist)):    
      bcsum += abs( cleanmetas[x][z][2] - cleanmetas[y][z][2] )
    row.append(bcsum /2 )    
  table.append(row)

g = open(outname, 'w')
g.write(',S1,S2,S4,S5,S5,S6,PRI,ASB,ANR\n')
print(',S1,S2,S4,S5,S5,S6,PRI,ASB,ANR\n')
for x in range(len(table)):
  outline = mglabel[x] +','
  for y in range(len(table[x]) -1):
    outline += str(table[x][y])[:6] + ','
  outline += str(table[x][len(table[x]) -1])[:6] + '\n'
  print(x, outline)
  g.write(outline)
g.close()  
print("\nDone!")
